In [1]:
#heheheh
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import shutil

# Caminho para a pasta com as imagens
dataset_path = 'flowers'
# Caminho para onde as subpastas serão criadas (opcional, se quiser em outro local)
output_path = 'flowers_reorganized'  # Modifique conforme necessário

# Crie a pasta de saída se ela não existir
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Lista de arquivos na pasta
image_files = os.listdir(dataset_path)

# Criação das subpastas e movimentação dos arquivos
for image_file in image_files:
    # Extrai a classe do nome do arquivo
    class_name = image_file.split('_')[0]
    
    # Cria a pasta da classe se ela não existir
    class_folder = os.path.join(output_path, class_name)
    if not os.path.exists(class_folder):
        os.makedirs(class_folder)
    
    # Move o arquivo para a pasta correspondente
    src_path = os.path.join(dataset_path, image_file)
    dst_path = os.path.join(class_folder, image_file)
    
    # Move o arquivo para a nova subpasta
    shutil.move(src_path, dst_path)

print("Reorganização concluída!")


Reorganização concluída!


In [2]:
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks

epochs = 100
batch_size = 32 # tamanho da separação do sgd, cada vez que roda ele atualiza os pessos da rede, entt ele vai atualizar os pesos 32 vezes, mtt pequeno é mais rápido mas instável
patience = 5 # quanto que o earlystop vai esperar para para, nesse caso são 5 epochs
learning_rate = 0.001 # alfa do SGD
model_path = 'kaggle/checkpoints/model.keras' # onde ele vai salvar o modelo, o 'dump'

In [5]:
num_classes = 10  # Define corretamente o número de classes

model = models.Sequential([
    # diminuir para fazer covuluções sem dar milhões de parâmetros
    layers.Resizing(56, 56), # colocando todas as imagens do mesmo tamanho, nn corta a imagem apenas diminui a resolução    
    layers.Rescaling(1.0/255), # deixar todos os valores dos pixels entre 0 e 1
    layers.RandomRotation((-0.2 , 0.2)), # leve rotação para cada imagem, rotacionar deixa mais lento mas aumenta a precisão


    # covolucao
    layers.Conv2D(32, (3, 3), # 32 filtros 3 por 3, tenho 32 imagens 54 por 54
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal() # valor aleatório com distribuição normal 
    ),
    
    layers.MaxPooling2D((2, 2)), # 32 imagens 27 por 27, pooling pega cada 4 quadradinhos e pega o valor máximo de cada quadradinho, isso diminui a lentidão

    layers.Conv2D(64, (4, 4),
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
                  
    layers.MaxPooling2D((2, 2)),
                  
    layers.Flatten(), # vai pegar tds as imagens 9 x 8 e tranformar num vetor gigante

    layers.Dropout(0.2), # desativa 20% dos dados na camada
                  
    layers.Dense(128, 
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),
                  
    layers.Dense(64, 
        activation = 'relu',
        kernel_initializer = initializers.RandomNormal()
    ),

    layers.Dense(num_classes, activation='softmax')  # Ajustado para múltiplas classes
])

In [6]:
model.compile(
    optimizer = optimizers.Adam( # adam é um algoritmo keras
        learning_rate = learning_rate
    ),
    # loss=losses.CategoricalCrossentropy(), # mais de uma classe, quando nn é um int a classe
    loss=losses.SparseCategoricalCrossentropy(), # quando é int a classe
    # metrics = [metrics.BinaryAccuracy(), metrics.Precision(), metrics.Recall() ] # para uma classe só, 1 ou 0
    metrics=['accuracy']  # Usa accuracy normal para multiclasse
)


In [8]:
train = utils.image_dataset_from_directory(
    output_path,
    validation_split = 0.2,
    subset= 'training',
    shuffle=True,
    seed=42,
    image_size=(244,244),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    output_path,
    validation_split = 0.2,
    subset= 'validation',
    shuffle=True,
    seed=42,
    image_size=(244,244),
    batch_size = batch_size
)

print(train.class_names)

# Se as imagens forem RGB, o formato precisa ser (batch_size, altura, largura, 3). Para conferir:

Found 733 files belonging to 10 classes.
Using 587 files for training.
Found 733 files belonging to 10 classes.
Using 146 files for validation.
['bougainvillea', 'daisies', 'garden', 'gardenias', 'hibiscus', 'hydrangeas', 'lilies', 'orchids', 'peonies', 'tulip']


In [9]:

    
model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience
        )
        # callbacks.ModelCheckpoint(
        #     filepath = model_path,
        #     save_weights_only = False,
        #     monitor = 'loss',
        #     mode = 'min',
        #     save_best_only = True
        # )
    ]
)

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - accuracy: 0.1386 - loss: 2.2751 - val_accuracy: 0.3425 - val_loss: 1.9819
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.2906 - loss: 1.9286 - val_accuracy: 0.2877 - val_loss: 1.8758
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.3520 - loss: 1.7667 - val_accuracy: 0.2260 - val_loss: 2.0255
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.3528 - loss: 1.7664 - val_accuracy: 0.3767 - val_loss: 1.6772
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.4365 - loss: 1.5977 - val_accuracy: 0.5000 - val_loss: 1.5866
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.4862 - loss: 1.4597 - val_accuracy: 0.3904 - val_loss: 1.7534
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.5117 - loss: 1.3978 - val_accuracy: 0.4521 - val_loss: 1.6529
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.5063 - loss: 1.4236 - val_accuracy: 0.